In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

import os #MAKDIR & DELETING FILES
import csv #COMMA SEPARATED VALUES LIBRARY

In [2]:
#TO WŁĄCZA STRONĘ
PATH = r"C:\Users\PC\1st\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/mazowieckie")

In [3]:
def create_folder_in_current_directory(folderName):#FOLDER NAME TO STRING - MA BYĆ W CUDZYSŁOWIE
    try:
        os.mkdir(folderName)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"'")
        
#DIRECTORY MA BYĆ Z UKOŚNIKAMI "/", A NIE "\"!!! POZA TYM OBA ARGUMENTY W CUDZYSŁOWACH
def create_folder_in_given_directory(directory, folderName):
    path = os.path.join(directory, folderName)
    try:
        os.mkdir(path)
    except:
        print("Nie stworzono folderu o nazwie: '" + folderName+"' w: " + path)

In [4]:
#TO TWOŻY LISTĘ LINKÓW "links".
posts = driver.find_elements_by_css_selector("a[data-cy='listing-item-link']")
links = []
#print("To jest lista ogłoszeń:")
for post in posts:
    links.append(post.get_attribute("href"))
    #print(links[posts.index(post)])#To jest tylko do wypisywania linków. Można usunąć.
print("Liczba ogłoszeń: " + str(len(links)))

Liczba ogłoszeń: 27


In [11]:
data = []
driver.get(links[0])#WEJŚĆ NA STRONĘ

In [12]:
table = driver.find_element_by_class_name("egzohkh2")
fields = table.find_elements_by_xpath(".//*")
for field in fields:
    if (fields.index(field)!=0):
        teksty = field.find_elements_by_xpath(".//*");
        if(len(teksty)==2):
            data.append(teksty[0].text)
            data.append(teksty[1].text)
    else:
        teksty = field.find_elements_by_xpath(".//*");#ZEBRAĆ WSZYSTKIE JEJ ELEMENTY (TYTUŁ I WARTOŚĆ, ZDJĘCIE Z JAKIEGOŚ POWODU SIE NIE ZNAJDUJE)
        if(len(teksty)==2):
            data.append(teksty[0].text)#WPISUJE WARTOŚCI DO TABLICY, Z KTÓREJ PÓŹNIEJ BĘDZIE WPISYWANE DO PLIKU
            temp = teksty[1].text
            temp = temp[:-1]
            data.append(temp)
print (data)

['Powierzchnia:', '111,07 m', 'Liczba pokoi:', '3', 'Rynek:', 'pierwotny', 'Rodzaj zabudowy:', 'szeregowiec', 'Piętro:', '1', 'Materiał budynku:', 'beton komórkowy', 'Okna:', 'plastikowe', 'Ogrzewanie:', 'gazowe', 'Rok budowy:', '2020', 'Stan wykończenia:', 'do wykończenia', 'Forma własności:', 'pełna własność']


In [13]:
with open('dane.csv', 'w', newline='') as csvfile:
    pisacz = csv.writer(csvfile, delimiter=';')
    j = 0
    i = 0
    if (len(data) > 2):
        for i in range(0, (int(len(data)/2))):
            try:
                pisacz.writerow([data[j],data[j+1]])
                j+=2
            except:
                print("Nie udało się coś")
    else:
        print("Za mało danych")

In [159]:
#WAŻNE NA PRZYSZŁOŚĆ
field = driver.find_element_by_css_selector("div[aria-label='Powierzchnia']")#ZNALEŹĆ POWIERZCHNIĘ
teksty = field.find_elements_by_xpath(".//*");#ZEBRAĆ WSZYSTKIE JEJ ELEMENTY (TYTUŁ I WARTOŚĆ, ZDJĘCIE Z JAKIEGOŚ POWODU SIE NIE ZNAJDUJE)
data.append(teksty[0].text)#WPISUJE WARTOŚCI DO TABLICY, Z KTÓREJ PÓŹNIEJ BĘDZIE WPISYWANE DO PLIKU
temp = teksty[1].text
temp = temp[:-1]
data.append(temp)
print(data)

['Powierzchnia:', '38 m']


In [160]:
field = driver.find_element_by_css_selector("div[aria-label='Forma własności']")
teksty = field.find_elements_by_xpath(".//*");
data.append(teksty[0].text)
data.append(teksty[1].text)